In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='sucessive',
        train_start=datetime(2020,2,1), train_end=datetime(2025,2,1),
        test_start=datetime(2022,2,1),  test_end=datetime(2023,2,1),
        search_type='clusters', search_subtype='kmeans', search_filter='',
        label_method='random', n_models=1, debug=False,
    ),
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='sucessive',
        train_start=datetime(2020,2,1), train_end=datetime(2025,2,1),
        test_start=datetime(2022,2,1),  test_end=datetime(2023,2,1),
        search_type='clusters', search_subtype='kmeans', search_filter='causal',
        label_method='random', n_models=1, debug=False,
    ),
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='sucessive',
        train_start=datetime(2020,2,1), train_end=datetime(2025,2,1),
        test_start=datetime(2022,2,1),  test_end=datetime(2023,2,1),
        search_type='clusters', search_subtype='kmeans', search_filter='mapie',
        label_method='random', n_models=1, debug=False,
    ),
]

# Crear tag para cada configuración
for cfg in configs:
    # Construir el tag asegurando que no haya dobles guiones bajos por campos vacíos
    tag_parts = [
        cfg['symbol'],
        cfg['timeframe'],
        cfg['direction'],
        cfg['label_method'][:2],
        cfg['search_type'][:3],
        (cfg.get('search_subtype') or '')[:2],
        (cfg.get('search_filter') or '')[:2],
    ]
    # Filtrar partes vacías y unir con "_"
    cfg["tag"] = "_".join([part for part in tag_parts if part]).strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None

n_trials = 5000
if study:
    n_trials = len(study.trials) + n_trials

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD_H1_buy_ra_clu_km_ca] modelo 0 trial 1/5000 score=-1.000000 avg=44.85s mem=408.62MB 
[XAUUSD_H1_buy_ra_clu_km] modelo 0 trial 1/5000 score=-1.000000 avg=46.73s mem=395.94MB 
[XAUUSD_H1_buy_ra_clu_km] modelo 0 trial 2/5000 score=0.441829 avg=41.74s mem=439.81MB 
[XAUUSD_H1_buy_ra_clu_km_ca] modelo 0 trial 2/5000 score=0.274012 avg=45.18s mem=443.59MB 
[XAUUSD_H1_buy_ra_clu_km] modelo 0 trial 3/5000 score=0.441829 avg=32.30s mem=441.99MB 
[XAUUSD_H1_buy_ra_clu_km_ma] modelo 0 trial 1/5000 score=-1.000000 avg=99.53s mem=461.15MB 
[XAUUSD_H1_buy_ra_clu_km_ca] modelo 0 trial 3/5000 score=0.274012 avg=37.38s mem=454.12MB 
[XAUUSD_H1_buy_ra_clu_km_ma] modelo 0 trial 2/5000 score=0.293010 avg=86.61s mem=482.17MB 
[XAUUSD_H1_buy_ra_clu_km] modelo 0 trial 4/5000 score=0.441829 avg=44.55s mem=407.87MB 
[XAUUSD_H1_buy_ra_clu_km_ca] modelo 0 trial 4/5000 score=0.274012 avg=50.58s mem=480.17MB 
[XAUUSD_H1_buy_ra_clu_km] modelo 0 trial 5/5000 score=0.441829 avg=41.34s mem=482.98MB 
[XAUUSD_H1_